## Jupyter Notebook para responder las preguntas de la Task 1

Lectura de los DataFrames:

In [1]:
from pathlib import Path    
import pandas as pd

RAW_DATA_PATH = Path('../../data/raw')

train_df = pd.read_csv(         RAW_DATA_PATH / 'train.csv')
test_df = pd.read_csv(          RAW_DATA_PATH / 'test.csv')
products_df = pd.read_pickle(   RAW_DATA_PATH / 'products.pkl')
users_df = pd.read_csv(         RAW_DATA_PATH / 'users_data.csv')

In [2]:
import ast

# Extraer datos del string (JSON) de la columna 'values'
def extract_data_from_string(df, column_name):
    
    df[column_name] = df[column_name].apply(ast.literal_eval)
    
    df['country'] = df[column_name].apply(lambda x: x['country'])
    df['R'] = df[column_name].apply(lambda x: x['R'])
    df['F'] = df[column_name].apply(lambda x: x['F'])
    df['M'] = df[column_name].apply(lambda x: x['M'])
    
    df = df.drop(columns=[column_name])
    
    df = df.explode(['country', 'R', 'F', 'M'])
    
    df['country'] = df['country'].astype(int)
    df['R'] = df['R'].astype(int)
    df['F'] = df['F'].astype(int)
    df['M'] = df['M'].astype(float)
    
    return df

users_df = extract_data_from_string(users_df, 'values')

#### **Task 1:** Answer the following questions and develop two functions about the train, clients and products datasets:
  - **Q1:** Which product (`partnumber`) with `color_id` equal to 3   belongs to the lowest `family` code with a `discount`? 

In [3]:
# Productos con descuento
discounted_products = products_df[(products_df['discount']==1)]

# Menor codigo de familia con descuento
min_fam_desc = discounted_products['family'].min()

# Productos cuyo codigo de familia es el menor con descuento
min_fam_desc_products = products_df[products_df['family']==min_fam_desc]

# Del subconjunto anterior, aquel cuyo color_id = 3
target = min_fam_desc_products[min_fam_desc_products['color_id']==3]

print(len(target))
print(target.head())
# Siendo estrictos con el enunciado, estos son los 34 productos que verifican tener color_id = 3 y pertenecer al menor código de familia con descuento (family = 1)

target_2 = products_df[(products_df['discount']==1) & (products_df['family']==min_fam_desc) & (products_df['color_id']==3)]
print(len(target_2))
print(target_2)
# Siendo menos estrictos, y entendiendo que la condición de tener descuento hay que aplicarla también sobre el resultado final, (discount=1), entonces tendríamos 2 productos:

# No existe un resultado único para la pregunta descrita, por lo que escogemos como resultado el primer partnumber 18091 , que verifica ambas interpretaciones de la pregunta.
Q1 = target_2.iloc[0]['partnumber']
print(Q1)

17
      discount                                          embedding  partnumber  \
8557         0  [-0.13718931, 0.29018262, -0.11451288, -0.1793...       39798   
8558         0  [-0.09487282, -0.14113918, -0.14696811, -0.191...       38018   
8559         0  [0.19879724, -0.21834055, -0.030612089, -0.054...       36259   
8561         0  [-0.07990676, -0.005529883, -0.13825129, -0.09...       38933   
19413        1  [-0.16633523, -0.13765946, 0.31306392, -0.1556...       17265   

       color_id  cod_section  family  
8557          3          4.0       1  
8558          3          4.0       1  
8559          3          4.0       1  
8561          3          4.0       1  
19413         3          4.0       1  
1
      discount                                          embedding  partnumber  \
19413        1  [-0.16633523, -0.13765946, 0.31306392, -0.1556...       17265   

       color_id  cod_section  family  
19413         3          4.0       1  
17265


  - **Q2:** In the country where most users have made purchases totaling less than 500 (`M`) , which is the user who has the lowest purchase frequency (`F`), the most recent purchase (highest `R`) and the lowest `user_id`? Follow the given order of variables as the sorting priority.

In [4]:
# Obtenemos el total por usuaro de M, F y R,
unique_users_df = users_df.groupby(['user_id', 'country']).agg({
    'M': 'sum',         # Sum purchases per country
    'F': 'sum',         # Sum frequency per country
    'R': 'max'          # Most recent per country
}).reset_index()

# Obtenemos el país con mayor número de usuarios que tienen < 500 en la columna M (compras)
unique_users_df['less_500_purchases'] = unique_users_df['M'] < 500
country_agg = unique_users_df.groupby('country', as_index=False)['less_500_purchases'].sum()
target_country = country_agg.loc[country_agg['less_500_purchases'].idxmax(), 'country']
print(f'Country with most users having purchases <500: {target_country}')

# Filtramos el país obtenido anteriormente y ordenamos los usuarios por F (menor a mayor), R (mayor a menor) y user_id (menor a mayor).
users_df_c = unique_users_df[unique_users_df['country'] == target_country]
sorted_user = users_df_c.sort_values(
    by=['F', 'R', 'user_id'], 
    ascending=[True, False, True]
).iloc[0]

Q2 = sorted_user['user_id']
print(Q2)

Country with most users having purchases <500: 25
187374


  - **Q3:** Among the products that were added to the cart at least once, how many times is a product visited before it is added to the cart in average? Give the answer with 2 decimals.

In [15]:
# Filtramos los productos que han sido añadidos al carrito al menos una vez
products_subset = train_df[train_df['add_to_cart']==True]['partnumber'].unique()
train_df_s = train_df[train_df['partnumber'].isin(products_subset)]

# Agrupamos y contamos registros (visitas o interacciones) y sumamos las veces que se ha añadido al carrito
product_stats = train_df_s.groupby('partnumber').agg({
    'add_to_cart': ['sum','count']
}).reset_index()
product_stats.columns = ['partnumber', 'cart_adds', 'total_visits']

In [17]:
# Cálculo
product_stats['visits_before_cart'] = (product_stats['total_visits'] - product_stats['cart_adds']) / product_stats['cart_adds']

Q3 = round(product_stats['visits_before_cart'].mean(),2) 
print(Q3)

22.69


In [47]:
# Get products added to cart and their first cart addition time
cart_additions = train_df[train_df['add_to_cart'] == 1].groupby('partnumber')['timestamp_local'].min()

# For these products, count visits up until first cart addition
result = (
    train_df.merge(
        cart_additions.reset_index(),
        on='partnumber',
        suffixes=('', '_first_cart')
    )
    .query('timestamp_local <= timestamp_local_first_cart')
    .groupby('partnumber')
    .size()
    .mean()
)

Q3 = round(result, 2)
print(Q3)

25.71


  - **Q4:** Which device (`device_type`) is most frequently used by users to make purchases (`add_to_cart` = 1) of discounted products (`discount` = 1)?

In [7]:
# Filtramos los datasets por separado y luego resolvemos el inner join
train_df_added_cart = train_df[train_df['add_to_cart']==True]
discounted_products_df = products_df[products_df['discount'] == 1]
result_df = train_df_added_cart.merge(discounted_products_df[['partnumber']], on='partnumber', how='inner')[['device_type']]

# Obtenemos la moda (valor más frecuente) de la columna 'device_type' en el resultado
Q4 = result_df['device_type'].mode()[0]
print(Q4)

1


  - **Q5:** Among users with purchase frequency (`F`) in the top 3 within their purchase country, who has interacted with the most products (`partnumber`) in sessions conducted from a device with identifier 3 (`device_type` = 3)?

In [8]:
# Obtener los 3 usuarios con mayor valor de F para cada valor de country
top_3_per_country = users_df.groupby('country').apply(lambda x: x.nlargest(3, 'F')).reset_index(drop=True)

# Filtramos los usuarios en el dataset de interacciones, y filtramos que sean sesiones en device_type = 3
target_users = top_3_per_country['user_id'].unique()
train_df_a = train_df[(train_df['user_id'].isin(target_users)) & (train_df['device_type'] == 3)]
result_df = train_df_a.groupby('user_id').agg({
    'partnumber' : 'nunique'
}).sort_values('partnumber', ascending=False)

# Obtenemos el user_id que ha interaccionado con más productos
Q5 = int(result_df.index[0])  
print(Q5)

C:\Users\chans\AppData\Local\Temp\ipykernel_19292\878451378.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_3_per_country = users_df.groupby('country').apply(lambda x: x.nlargest(3, 'F')).reset_index(drop=True)


72153


  - **Q6:** For interactions that occurred outside the user's country of residence, how many unique family identifiers are there? Take into account any registered country for each user, as there may be more than one country per user.

In [9]:
# Hacemos Join con los usuarios y sus países de residencia y nos quedamos con las filas que no cruzan
train_df_b = train_df[train_df['user_id'].notna()].merge(users_df[['user_id', 'country']], 
                                on=['user_id', 'country'], 
                                how='left',
                                indicator=True)

print(len(train_df_b))

train_df_b = train_df_b[train_df_b['_merge'] == 'left_only'].drop('_merge', axis=1)

print(len(train_df_b))

6856730
14257


In [10]:
train_df_c = train_df_b.merge(products_df[['partnumber', 'family']], 
                                on=['partnumber'], 
                                how='left')

In [11]:
Q6 = train_df_c['family'].nunique() 
print(Q6)

97


  - **Q7:** Among interactions from the first 7 days of June, which is the most frequent page type where each family is added to the cart? Return it in the following format: `{'('family'): int('most_frequent_pagetype')}` . In case of a tie, return the smallest pagetype.

In [12]:
train_df_d = train_df[(train_df['date'] >= '2024-06-01') & (train_df['date'] <= '2024-06-07') & (train_df['add_to_cart'] == 1)]

In [13]:
train_df_e = train_df_d.merge(products_df[['partnumber', 'family']], 
    on=['partnumber'], 
    how='inner')

# Agrupar por familia y tipo de pagina y contar las apariciones, ordenar descendente
train_df_f = train_df_e.groupby(['family','pagetype']).size().reset_index(name='count')
train_df_f = train_df_f.sort_values('count', ascending=False)

# Crear el diccionaro objetivo
Q7 = {}
for family in sorted(train_df_f['family'].unique()):
    family_data = train_df_f[train_df_f['family'] == family]
    # Ordenar por apariciones descendente, y por pagetype ascendente (caso de empate)
    most_common = family_data.sort_values(['count', 'pagetype'], ascending=[False, True]).iloc[0]
    Q7[str(family)] = int(most_common['pagetype'])

print(Q7)

{'1': 24, '2': 24, '3': 24, '4': 24, '5': 24, '6': 24, '7': 24, '8': 24, '9': 24, '10': 24, '11': 24, '12': 24, '13': 24, '14': 24, '15': 24, '16': 24, '17': 24, '18': 24, '19': 24, '21': 24, '22': 24, '23': 24, '24': 24, '26': 24, '27': 24, '28': 24, '29': 24, '30': 24, '31': 24, '32': 24, '33': 24, '34': 24, '35': 24, '36': 24, '37': 24, '38': 24, '40': 24, '41': 24, '42': 24, '43': 24, '44': 24, '45': 24, '46': 24, '47': 24, '48': 24, '49': 24, '50': 24, '51': 24, '52': 24, '53': 24, '54': 24, '56': 24, '57': 24, '58': 24, '59': 24, '60': 24, '61': 24, '62': 24, '63': 24, '64': 24, '65': 24, '66': 24, '67': 24, '68': 24, '69': 24, '70': 24, '71': 24, '72': 24, '73': 24, '74': 24, '75': 24, '76': 24, '77': 24, '78': 24, '79': 24, '81': 24, '82': 24, '83': 24, '84': 24, '85': 24, '86': 24, '87': 24, '88': 24, '89': 24, '90': 24, '91': 24, '92': 24, '93': 24, '94': 24, '95': 24, '96': 24, '97': 24, '99': 24, '100': 24, '101': 24, '102': 24, '103': 24, '104': 24, '105': 24, '106': 24, '

Entregamos en el formato deseado:

In [14]:
import json
import os
import numpy as np

# Function to convert numpy types to native Python types
def convert_to_serializable(obj):
    if isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    return obj

# Create predictions dictionary
predictions = {
    "target": {
        "query_1": {
            "partnumber": int(Q1)
        },
        "query_2": {
            "user_id": int(Q2)
        },
        "query_3": {
            "average_previous_visits": float(Q3)
        },
        "query_4": {
            "device_type": int(Q4)
        },
        "query_5": {
            "user_id": int(Q5)
        },
        "query_6": {
            "unique_families": int(Q6)
        },
        "query_7": convert_to_serializable(Q7) 
    }
}

# Create predictions directory if it doesn't exist
os.makedirs('predictions', exist_ok=True)

# Save to JSON file with proper formatting
with open('../../predictions/predictions_1.json', 'w') as f:
    json.dump(predictions, f, indent=4)